In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

In [11]:
#df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/blogtext.csv")
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [92]:

df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/blogtext.csv")

In [80]:
df.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [93]:
df=df.iloc[0:20000]

In [82]:
df.head(5)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [65]:
df.shape

(20000, 7)

In [94]:
df['age'] = df['age'].astype(str)
df['id'] = df['id'].astype(str)

In [95]:

df=df.applymap(lambda s:s.lower())

#select only numbers and alphabets
import re

df=df.applymap(lambda s: re.sub('[^0-9a-z #+_]'," ",s))

#strip
df=df.applymap(lambda s:s.strip())

In [96]:
df['text'].head()

0    info has been found  +   100 pages  and 4 5 mb...
1    these are the team members    drewes van der l...
2    in het kader van kernfusie op aarde   maak je ...
3                                  testing     testing
4    thanks to yahoo  s toolbar i can now  capture ...
Name: text, dtype: object

In [97]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
#df.text = df.text.apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))
#df.text=df.text.apply(lambda x: [item for item in x if item not in stopwords])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [98]:
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stopwords]
    return ' '.join(no_stopword_text)

df['text'] = df['text'].apply(lambda x: remove_stopwords(x))

In [99]:
df['text'].head()

0    info found + 100 pages 4 5 mb pdf files wait u...
1    team members drewes van der laag urllink mail ...
2    het kader van kernfusie op aarde maak je eigen...
3                                      testing testing
4    thanks yahoo toolbar capture urls popups means...
Name: text, dtype: object

In [75]:
df.shape

(20000, 7)

In [102]:

#df['labels']=df[['gender','age','topic','sign']].apply(lambda x:','.join(x), axis = 1) 
df['labels']=' '
for w in range(0,df.shape[0]):
    blog_label=[]
    blog_label.append(df['gender'][w])
    blog_label.append(df['age'][w])
    blog_label.append(df['topic'][w])
    blog_label.append(df['sign'][w])
    df['labels'][w]=blog_label


In [103]:
df['labels'].head()

0                   [male, 15, student, leo]
1                   [male, 15, student, leo]
2                   [male, 15, student, leo]
3                   [male, 15, student, leo]
4    [male, 33, investmentbanking, aquarius]
Name: labels, dtype: object

In [104]:
merged_data=df.drop(labels =['date','gender', 'age','topic','sign','id'], axis = 1)
merged_data.head()

,text,labels
0,info found + 100 pages 4 5 mb pdf files wait u...,"[male, 15, student, leo]"
1,team members drewes van der laag urllink mail ...,"[male, 15, student, leo]"
2,het kader van kernfusie op aarde maak je eigen...,"[male, 15, student, leo]"
3,testing testing,"[male, 15, student, leo]"
4,thanks yahoo toolbar capture urls popups means...,"[male, 33, investmentbanking, aquarius]"


In [105]:
X = df['text']
y=df['labels']

In [122]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.33, random_state = 43)

In [126]:
from sklearn.feature_extraction.text import CountVectorizer
vect=CountVectorizer(ngram_range=(1, 2))


In [127]:
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)


In [128]:
X_test_dtm = vect.transform(X_test)

In [129]:
print(X_train_dtm.shape)
print(X_test_dtm.shape)

(13400, 953029)
(6600, 953029)


In [120]:
multilabel_binarizer = MultiLabelBinarizer()
y_train_mlb=multilabel_binarizer.fit_transform(y_train)
y_test_mlb=multilabel_binarizer.transform(y_test)


In [123]:
print(X_train.shape)
print(X_test.shape)
print(y_train_mlb.shape)
print(y_test_mlb.shape)


(13400,)
(6600,)
(13400, 76)
(6600, 76)


In [124]:
multilabel_binarizer.classes_

array(['13', '14', '15', '16', '17', '23', '24', '25', '26', '27', '33',
       '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44',
       '45', '46', '47', '48', 'accounting', 'advertising', 'agriculture',
       'aquarius', 'architecture', 'aries', 'arts', 'automotive',
       'banking', 'biotech', 'businessservices', 'cancer', 'capricorn',
       'communications media', 'construction', 'consulting', 'education',
       'engineering', 'fashion', 'female', 'gemini', 'government',
       'humanresources', 'indunk', 'internet', 'investmentbanking', 'law',
       'lawenforcement security', 'leo', 'libra', 'male', 'manufacturing',
       'marketing', 'military', 'museums libraries', 'non profit',
       'pisces', 'publishing', 'realestate', 'religion', 'sagittarius',
       'science', 'scorpio', 'sports recreation', 'student', 'taurus',
       'technology', 'telecommunications', 'transportation', 'virgo'],
      dtype=object)

In [130]:
df['labels'].to_dict()

In [131]:
gender=[]
age=[]
topic=[]
sign=[]

In [132]:
for pattern in df['labels']:
    w=0
    for value in pattern:
        if w==0:
            gender.append(value)
        elif w==1:
            age.append(value)
        elif w==2:
            topic.append(value)
        elif w==3:
            sign.append(value)
        w+=1


In [134]:
from collections import Counter
age_freq=Counter(age)
gender_freq=Counter(gender)
topic_freq=Counter(topic)
sign_freq=Counter(sign)


In [135]:
def dict_print (age,gender,topic,sign ):
    w = age.copy()   
    w.update(gender)    
    w.update(topic)
    w.update(sign)
    return w


In [136]:
dict_print(age_freq,gender_freq,topic_freq,sign_freq)

Counter({'13': 113,
         '14': 811,
         '15': 1097,
         '16': 1236,
         '17': 1961,
         '23': 1963,
         '24': 1557,
         '25': 1190,
         '26': 919,
         '27': 2320,
         '33': 769,
         '34': 871,
         '35': 2494,
         '36': 1726,
         '37': 130,
         '38': 85,
         '39': 105,
         '40': 1,
         '41': 82,
         '42': 47,
         '43': 6,
         '44': 9,
         '45': 72,
         '46': 188,
         '47': 8,
         '48': 240,
         'accounting': 35,
         'advertising': 42,
         'agriculture': 46,
         'aquarius': 1313,
         'architecture': 45,
         'aries': 5209,
         'arts': 358,
         'automotive': 111,
         'banking': 89,
         'biotech': 36,
         'businessservices': 184,
         'cancer': 1536,
         'capricorn': 930,
         'communications media': 414,
         'construction': 21,
         'consulting': 166,
         'education': 759,
         'engi

In [137]:
clf=LogisticRegression(solver='lbfgs')
clf=OneVsRestClassifier(clf)


In [ ]:
clf.fit(X_train_dtm,y_train_mlb)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [ ]:
y_pred_test=clf.predict(X_test_dtm)

In [ ]:
accuracy_score(y_test_mlb, y_pred_test)

In [ ]:
f1_score(y_test_mlb, y_pred_test,average='micro')

In [ ]:
recall_score(y_test_mlb,y_pred_test,average='micro')

In [ ]:
precision_score(y_test_mlb,y_pred_test,average='micro')

In [ ]:
multilabel_binarizer.inverse_transform(y_pred_test)[0:5]

In [ ]:
y_test[0:5]

In [ ]:
y_pred_prob_test = clf.predict_proba(X_test_dtm)

In [ ]:
y_pred_new_test = (y_pred_prob_test >= t).astype(int)

In [ ]:
accuracy_score(y_test_mlb, y_pred_new_test)

In [ ]:
f1_score(y_test_mlb, y_pred_new_test, average="micro")

In [ ]:
recall_score(y_test_mlb,y_pred_new_test,average='micro')

In [ ]:
precision_score(y_test_mlb,y_pred_new_test,average='micro')

In [ ]:
multilabel_binarizer.inverse_transform(y_pred_new_test)[0:5]

In [ ]:
y_test[0:5]